In [ ]:
import requests
import json
import pandas as pd
from pandas import json_normalize
from google.colab import files

token='7752ee848bac741a1b2ccb1be43a19fd'
AV_token="0b8e6db6509f2bf80bda79ad71f5a6b6"

def get_project_id(proj_name,i=token):
    url = "https://api.awrcloud.com/api/marketshare/v1/endpoint.php?action=getProjects&token={}".format(i)
    response = requests.get(url).json()
    ID_df = json_normalize(response)
    proj_id = ID_df.iloc[list(ID_df.iloc[:,1]).index(proj_name),0]
    return proj_id
#test1 = get_project_id("LG.com_UK")

def get_search_engines_id(proj_id,mytoken=token):
    url = "https://api.awrcloud.com/api/marketshare/v1/endpoint.php?action=getSearchEngines&token={}&projectId={}".format(mytoken,proj_id)
    response = requests.get(url).json()
    se_df = json_normalize(response)
    id = list(se_df['id']).pop()
    return id

def get_search_engines_name(proj_id,mytoken=token):
    url = "https://api.awrcloud.com/api/marketshare/v1/endpoint.php?action=getSearchEngines&token={}&projectId={}".format(mytoken,proj_id)
    response = requests.get(url).json()
    se_df = json_normalize(response)
    id = list(se_df['name']).pop()
    return id

def get_keyword_group_df(proj_id=111, i=token):
    url = "https://api.awrcloud.com/api/marketshare/v1/endpoint.php?action=getKeywordGroups&token={}&projectId={}".format(i,proj_id)
    response = requests.get(url).json()
    group_df = json_normalize(response)
    return group_df
#test2 = getkeyword_group_df(test1)

def get_keyword_group_id(group_name, proj_id=111, i=token):
    url = "https://api.awrcloud.com/api/marketshare/v1/endpoint.php?action=getKeywordGroups&token={}&projectId={}".format(i,proj_id)
    response = requests.get(url).json()
    group_df = json_normalize(response)
    group_id = group_df.iloc[list(group_df.iloc[:,1]).index(group_name),0]
    return group_id
#test3 = get_keyword_group_id('Brand',test1)

def get_metadata_df(proj_id,group_id,obj_date,i=token):
    meta_df = pd.DataFrame({})
    url = "https://api.awrcloud.com/api/marketshare/v1/endpoint.php?action=getMetadata&token={}&projectId={}&searchEngineId={}&keywordGroupId={}&startDate={}&stopDate={}&mode=plain".format(i,proj_id,-1,group_id,obj_date,obj_date)
    print(group_id)
    response = requests.get(url).text
    domain_list_raw = response.split('<br/>')
    colum_name = domain_list_raw[0].split(',')
    columindex=0
    for x in colum_name:
        flashdata= []
        for i in domain_list_raw[1:-1]:
            flashdata.append(i.split(',')[columindex])
        meta_df[x]=flashdata
        columindex = columindex+1
    return meta_df
#test4 = get_metadata_df(proj_id='111',group_id='33',obj_date='2021-03-29',i=token)
#test4 = get_metadata_df(test1,test3,'2021-03-30')

def get_domain_urls(proj_id,group_id,obj_date,obj_domain,i=token):
    url = "https://api.awrcloud.com/api/marketshare/v1/endpoint.php?action=getDomainUrls&token={}&projectId={}&searchEngineId=-1&keywordGroupId={}&date={}&domain={}&mode=plain".format(i,proj_id,group_id,obj_date,obj_domain)
    response = requests.get(url).text
    url_list_raw = response.split('<br/>')
    colum_name = url_list_raw[0].split(',')
    url_index=colum_name.index('url')
    colum_count=len(colum_name)
    url_df = pd.DataFrame(columns = colum_name)
    for i in url_list_raw[1:-1]:
        flashdata= []
        row_list = i.split(',')
        flashdata = flashdata+row_list[:url_index]
        flashdata.append(','.join(row_list[url_index:url_index-colum_count+1]))
        flashdata = flashdata+row_list[url_index-colum_count+1:]
        url_df = url_df.append(pd.Series(flashdata,index=url_df.columns), ignore_index=True)
    url_df=url_df.astype({'market share':'float','estimated visits':'float','click share':'float'})
    return url_df
#test5 = get_domain_urls(proj_id='111',group_id='33',obj_date='2021-03-29',obj_domain='lg.com',i=token)


def export_top_sites(av_proj_name,start,stop,searchE, num_url=12,token = AV_token):
    proj_name = '+'.join(av_proj_name.split())
    url = "https://api.awrcloud.com/v2/get.php?action=topsites_export&project={}&token={}&startDate={}&stopDate={}&pixelPosition=false&topUrls={}&searchEngine={}".format(proj_name, token, start, stop,num_url,searchE)
    response1 = requests.get(url).json()
    res_df = json_normalize(response1)
    down_url = res_df['details'][0]
    return down_url

 # 상위 노출 컨텐츠 추출 함수_키워드 추출
def export_top_sites_group(proj_name,  start_date, end_date, kwg_id, num_url=12, token=AV_token):
    url = "https://api.awrcloud.com/v2/get.php?action=topsites_export&project={}&token={}&startDate={}&stopDate={}&keywordGroupId={}&pixelPosition=true&sponsored=False&topUrls={}&xlsx=true".format(proj_name, token, start_date, end_date, str(kwg_id), num_url) 
    response = requests.get(url).json()
    return response


 # 상위 노출 컨텐츠 추출 함수
def export_top_sites(proj_name, start_date, end_date, num_url=12, token=AV_token): 
    url = "https://api.awrcloud.com/v2/get.php?action=topsites_export&project={}&token={}&startDate={}&stopDate={}&pixelPosition=true&sponsored=False&topUrls={}&xlsx=true".format(proj_name, token, start_date, end_date, num_url)
    response = requests.get(url).json()
    return response




In [ ]:
# # 입력부
insert_name = "LG.com_Spain"
select_date = "2021-06-13"

In [ ]:
# Market Share 키워드 그룹별 세부 url 리스트 추출(시트별) 
proj_id_re = get_project_id(insert_name)
group_name = get_keyword_group_df(proj_id_re)['name']
group_id = get_keyword_group_df(proj_id_re)['id']
filename = "{}_MarketShare_{}".format(insert_name,select_date)
count = 1
with pd.ExcelWriter('{}.xlsx'.format(filename)) as writer:
    for x in group_id[1:]:
        all_df = pd.DataFrame({})
        sub_df= get_metadata_df(proj_id = proj_id_re,group_id = x, obj_date = select_date,i=token)
        # all_df = pd.concat([all_df,sub_df.iloc[:,5:]])
        b = sub_df['domain']
        for y in b:
            try:
                sub_df2 = get_domain_urls(proj_id_re,x,select_date,y,token)
                all_df = pd.concat([all_df,sub_df2.iloc[:,5:]])
                print("{} 도메인".format(y))
            except:
                exception_flash = [y for ii in range(len(all_df.columns))]
                all_df = all_df.append(pd.Series(exception_flash,index=all_df.columns), ignore_index=True)
        all_df.to_excel(writer,sheet_name=group_name[count],index=True)
        count=count+1
files.download('{}.xlsx'.format(filename))

In [ ]:
# AWR 프로젝트 별 등록 키워드셋 리스트 추출(시트별) 
filename = "AWR_keywordset_{}".format(select_date)
url = "https://api.awrcloud.com/v2/get.php?action=projects&token={}".format(token)
response = requests.get(url).json()
pj_df = json_normalize(response['projects'])
is_LGcom_pj = pj_df['name'].str.contains("LG.com_")
is_daily_pj = pj_df['frequency'] == 'daily'
obj_pj_df = pj_df[is_LGcom_pj & is_daily_pj]
group_1_list = ['Generic','Core','Brand','Samsung Core','Samsung Brand']
with pd.ExcelWriter('{}.xlsx'.format(filename)) as writer:
    obj_pj_df.to_excel(writer,sheet_name="project_list",index=True)
    for y in obj_pj_df['name']:
        print(y)
        url = "https://api.awrcloud.com/v2/get.php?action=details&project={}&token={}".format(y,token)
        response2 = requests.get(url).json()
        a = response2.keys()
        kw_detail_df = json_normalize(response2['keywords'])
        # try:
        dicc_v1 = [','.join(list(set(x)&set(group_1_list))) for x in kw_detail_df['kw_groups']]
        dicc_v2 = [','.join(list(set(x) - set(group_1_list))) for x in kw_detail_df['kw_groups']]
        dicc_k = [y for y in kw_detail_df['name']]
        # except:
        #     dicc_v1 =['fail','fail']
        #     dicc_v2=['fail','fail']
        #     dicc_k=['fail','fail']
        kwdicc_df = pd.DataFrame({'group_1':dicc_v1,'group_2':dicc_v2},index=dicc_k,dtype=str)
        kwdicc_df.to_excel(writer,sheet_name= y,index=True)
files.download('{}.xlsx'.format(filename))

In [ ]:
# AWR 프로젝트 별 등록 컴피티터 리스트 추출(시트별) 
filename = "AWR_competitors_{}".format(select_date)
url = "https://api.awrcloud.com/v2/get.php?action=projects&token={}".format(token)
response = requests.get(url).json()
pj_df = json_normalize(response['projects'])
is_LGcom_pj = pj_df['name'].str.contains("LG.com_")
is_daily_pj = pj_df['frequency'] == 'daily'
obj_pj_df = pj_df[is_LGcom_pj & is_daily_pj]
group_1_list = ['Generic','Core','Brand','Samsung Core','Samsung Brand']
with pd.ExcelWriter('{}.xlsx'.format(filename)) as writer:
    obj_pj_df.to_excel(writer,sheet_name="project_list",index=True)
    for y in obj_pj_df['name']:
        url = "https://api.awrcloud.com/v2/get.php?action=details&project={}&token={}".format(y,token)
        response2 = requests.get(url).json()
        a = response2.keys()
        website_df = json_normalize(response2['websites'])
        website_df.to_excel(writer,sheet_name= y,index=True)
files.download('{}.xlsx'.format(filename))

In [ ]:
# Market Share 키워드 그룹별 세부 url 개수 count (일자 번들 입력) 
date_list = pd.date_range('2021-05-24', periods=7)
fail_df= pd.DataFrame({},columns =['f_project','f_group','f_date'])
proj_list = []

for xxx in proj_list:
    proj_id_re = get_project_id(xxx)
    print(proj_id_re)
    group_df = get_keyword_group_df(proj_id_re)

    # Keyword Group 대상 설정 부분
    is_general_cate = group_df['name'] == 'General - Category'
    # is_general_cate = group_df['name'] == 'General - Category'    
    group_select = group_df[is_general_cate]

    group_select.reset_index(inplace=True)
    group_select
    group_name = group_select['name']
    group_id = group_select['id']
    for x in group_id[:]:
        all_df = pd.DataFrame({})
        count = 0
        filename = "{}_result_{}".format(xxx,x)
        for date in date_list:
            select_date = str(date)[:10]
            print(select_date)
            try:
                sub_df= get_metadata_df(proj_id = proj_id_re,group_id = x, obj_date = str(select_date),i=token)
                sub_df = sub_df.astype({'urls': 'float'})
            except:
                flash_data = [xxx, x, select_date]
                fail_df = fail_df.append(pd.Series(flash_data, index=fail_df.columns), ignore_index=True)
                print(fail_df)
                sub_df = pd.DataFrame({})
                pass
            all_df = pd.concat([all_df,sub_df])
        grouped_df = all_df['urls'].groupby(all_df['domain'])
        aa = grouped_df.sum()
        bb = aa.sort_values(ascending=False)
        count=count+1
        bb.to_excel('{}.xlsx'.format(filename),index=True)
        print(count)
    files.download('{}.xlsx'.format(filename))
    print(fail_df)